In [21]:
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

require("data.table")
require("rpart")

#------------------------------------------------------------------------------
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 

particionar  <- function( data,  division, agrupa="",  campo="fold", start=1, seed=NA )
{
  if( !is.na(seed) )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x )} ,   division,  seq( from=start, length.out=length(division) )  ) )  

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
          by= agrupa ]
}
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

# Establezco el Working Directory
setwd(".")

# Cargo los datos
dataset  <- fread("../datasets/dataset_pequeno.csv")

#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]

#particiono estratificadamente el dataset
#Cambiar por la primer semilla de cada uno !

# Lista de semillas a probar
semillas <- c(102191, 677213, 727817, 311237, 660719, 106427)

semilla <- 106427
particionar(dataset, division = c(7, 3), agrupa = "clase_ternaria", seed = semilla)


param_basicos  <- list( "cp"=         -1,  #complejidad minima
                        "minsplit"=  400,  #minima cantidad de registros en un nodo para hacer el split
                        "minbucket"=  10,  #minima cantidad de registros en una hoja
                        "maxdepth"=    8 ) #profundidad máxima del arbol

#genero el modelo
modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                 data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                 xval= 0,
                 control=  param_basicos )  #aqui van los parametros


#aplico el modelo a los datos de testing
prediccion  <- predict( modelo,   #el modelo que genere recien
                        dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                        type= "prob") #type= "prob"  es que devuelva la probabilidad

#prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
#cada columna es el vector de probabilidades 

#agrego una columna que es la de las ganancias
dataset[  , ganancia :=  ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ) ]

#para testing agrego la probabilidad
dataset[ fold==2 , prob_baja2 := prediccion[, "BAJA+2"] ]

#calculo la ganancia en testing  qu es fold==2
ganancia_test  <- dataset[ fold==2 & prob_baja2 >  0.025, sum(ganancia) ]

#escalo la ganancia como si fuera todo el dataset
ganancia_test_normalizada  <-  ganancia_test / 0.3

estimulos  <- dataset[ fold==2 & prob_baja2 > 0.025 , .N ]
aciertos   <- dataset[ fold==2 & prob_baja2 > 0.025 & clase_ternaria =="BAJA+2", .N ]


cat( "Testing total: ",  dataset[ fold==2, .N ], "\n" )
cat( "Testing BAJA+2: ", dataset[ fold==2 & clase_ternaria =="BAJA+2", .N ], "\n" )  

cat( "Estimulos: ", estimulos, "\n" )
cat( "Aciertos (BAJA+2): ",  aciertos,  "\n" )

# Imprimir la semilla actual y la ganancia en testing (normalizada)
cat("Semilla: ", semilla, "\n")
cat("Ganancia en testing (normalizada): ", ganancia_test_normalizada, "\n\n")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,715327,38.3,1403282,75.0,1337187,71.5
Vcells,1402341,10.7,71112958,542.6,146947290,1121.2


Testing total:  49403 
Testing BAJA+2:  380 
Estimulos:  3441 
Aciertos (BAJA+2):  219 
Semilla:  106427 
Ganancia en testing (normalizada):  53190000 



In [ ]:
Semilla:  102191 
Ganancia en testing (normalizada):  58760000 

In [ ]:
Semilla:  677213 
Ganancia en testing (normalizada):  63180000 

In [ ]:
Semilla:  727817 
Ganancia en testing (normalizada):  57200000 

In [ ]:
Semilla:  311237 
Ganancia en testing (normalizada):  53940000 

In [ ]:
Semilla:  660719 
Ganancia en testing (normalizada):  50360000

In [ ]:
Semilla:  106427 
Ganancia en testing (normalizada):  53190000 